### Classification Network - All Embeddings - One-Hot grouping

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, \
CategoricalHinge, BinaryFocalCrossentropy,\
MeanSquaredError, LogCosh, CosineSimilarity, Huber, MeanSquaredLogarithmicError, MeanAbsoluteError

import pandas as pd
import pickle
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score

import matplotlib.pyplot as plt

C:\Users\dlezo\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# tf.config.list_physical_devices('GPU')

#### Load the dataset

In [3]:
seen = pd.read_csv('./Data/C56_71_TWOSIDES_one_hot_train.csv')
unseen = pd.read_csv('./Data/C56_71_TWOSIDES_one_hot_test.csv')

selected_classes = [
 'class_00', 'class_01', 'class_02', 'class_03', 'class_04', 'class_05', 'class_06', 'class_07', 'class_08', 'class_09',
 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17',             'class_19',
 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29',
 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39',
 'class_40', 'class_41', 'class_42', 'class_43', 'class_44',             'class_46', 'class_47', 'class_48', 'class_49',
 'class_50', 'class_51', 'class_52', 'class_53', 'class_54', 'class_55'
]
unseen_classes = ['class_18', 'class_45']
class_columns = selected_classes
unseen_columns = unseen_classes


print('Total pairs:', seen.shape[0])
for column in class_columns:
    print('Has/Doesn\'t have', column, seen[seen[column] == 1.0].shape[0], '/', seen[seen[column] == 0.0].shape[0])
        
seen    

Total pairs: 155137
Has/Doesn't have class_00 100000 / 55137
Has/Doesn't have class_01 46981 / 108156
Has/Doesn't have class_02 50429 / 104708
Has/Doesn't have class_03 13274 / 141863
Has/Doesn't have class_04 53738 / 101399
Has/Doesn't have class_05 19342 / 135795
Has/Doesn't have class_06 54162 / 100975
Has/Doesn't have class_07 28799 / 126338
Has/Doesn't have class_08 29005 / 126132
Has/Doesn't have class_09 31261 / 123876
Has/Doesn't have class_10 54575 / 100562
Has/Doesn't have class_11 51810 / 103327
Has/Doesn't have class_12 47356 / 107781
Has/Doesn't have class_13 52445 / 102692
Has/Doesn't have class_14 20528 / 134609
Has/Doesn't have class_15 52627 / 102510
Has/Doesn't have class_16 45863 / 109274
Has/Doesn't have class_17 40630 / 114507
Has/Doesn't have class_19 17600 / 137537
Has/Doesn't have class_20 53792 / 101345
Has/Doesn't have class_21 44431 / 110706
Has/Doesn't have class_22 13823 / 141314
Has/Doesn't have class_23 52057 / 103080
Has/Doesn't have class_24 42749 / 112

,drug1,drug2,class_00,class_01,class_02,class_03,class_04,class_05,class_06,class_07,...,class_46,class_47,class_48,class_49,class_50,class_51,class_52,class_53,class_54,class_55
0,55681,1243041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,134547,6313,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,4094,47858,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8627,3920,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,797195,41493,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155132,4448,32675,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
155133,4821,4816,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
155134,1998,7781,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
155135,20220,7398,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Load the Embeddings

In [4]:
with open('./Data/Embeddings/biobert_embeddings_128_pca.pkl', 'rb') as file:
    biobert = pickle.load(file)    
with open('./Data/Embeddings/chemberta_embeddings_128_pca.pkl', 'rb') as file:
    chemberta = pickle.load(file)    
with open('./Data/Embeddings/kg_bio2rdf_embeddings_128_pca.pkl', 'rb') as file:
    bio2rdf = pickle.load(file)    
with open('./Data/Embeddings/ssp_embeddings_128_pca.pkl', 'rb') as file:
    ssp = pickle.load(file)    

drugs = set(np.concatenate([
    seen['drug1'].unique(), seen['drug2'].unique(), unseen['drug1'].unique(), unseen['drug2'].unique()
]))
embeddings = {}
for rx_norm_id in list(drugs):
    embeddings[rx_norm_id] = np.concatenate([
        biobert[rx_norm_id], chemberta[rx_norm_id], bio2rdf[rx_norm_id], ssp[rx_norm_id]
    ])

seen['embedded'] = seen.apply(lambda x: pd.Series(
    [np.concatenate([embeddings[x['drug1']], embeddings[x['drug2']]])], 
    index=['embedded']), axis=1)
print('Seen shape:', seen.shape)
unseen['embedded'] = unseen.apply(lambda x: pd.Series(
    [np.concatenate([embeddings[x['drug1']], embeddings[x['drug2']]])], 
    index=['embedded']), axis=1)
print('Unseen shape:', unseen.shape)
seen.head()

Seen shape: (155137, 59)
Unseen shape: (103560, 6)


,drug1,drug2,class_00,class_01,class_02,class_03,class_04,class_05,class_06,class_07,...,class_47,class_48,class_49,class_50,class_51,class_52,class_53,class_54,class_55,embedded
0,55681,1243041,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[2.168276786804199, 1.0936528444290161, 0.9610..."
1,134547,6313,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,"[2.232297420501709, 0.9437393546104431, 0.0993..."
2,4094,47858,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.5903611183166504, -1.876413106918335, 1.70..."
3,8627,3920,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[-0.37425464391708374, 1.5704320669174194, -0...."
4,797195,41493,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,"[3.0385594367980957, 0.587708055973053, 0.8335..."


#### Load and Split Dataset

In [5]:
size = len(seen['embedded'][0])
# Get all the target columns from seen
target_columns = seen.columns[pd.Series(seen.columns).str.startswith('class_')]
targets = len(target_columns)
print('input size:', size, 'output size:', targets)

random_seed = 1 #42 #73 # 71

x_train, x_test, y_train, y_test = train_test_split (
    np.concatenate(seen['embedded'].values).reshape(-1, size), 
    seen[target_columns].values, test_size=1, random_state=random_seed)

input size: 1024 output size: 54


#### Experiment function

In [6]:
def run_experiment(name, model_size, dropout_rate, in_size, targets, \
                   activation_function, loss_function, learning_rate, metric, \
                   x_train, y_train, x_test, y_test, 
                   epochs, batch_size, validation_split, verbose=0\
                  ):
    # Creating model using the Sequential in tensorflow
    model = Sequential([
                Input(shape=(in_size,)),
                Dense(in_size//model_size, activation='relu'),
                Dropout(dropout_rate),
                Dense(in_size//model_size, activation='relu'),
                Dropout(dropout_rate),
                Dense(in_size//model_size, activation='relu'),
                Dropout(dropout_rate),
                Dense(in_size//model_size, activation='relu'),
                Dropout(dropout_rate),
                Dense(in_size//model_size//4, activation='relu'),
                Dropout(dropout_rate),
                Dense(targets, activation=activation_function)
            ])
    model.compile(
        loss=loss_function,
        optimizer=Adam(learning_rate=learning_rate),
        metrics = [metric]
    )
    if verbose > 0:
        print(model.summary())
    filename = name+'_'+loss_function.name+'_'+str(learning_rate)+'_'+\
               metric+'_'+str(epochs)+'_'+str(batch_size)+'_'+\
               str(validation_split)+'_'+str(model_size)+'_'+\
               str(dropout_rate)+'_'+str(in_size)+'_'+str(targets)+\
               '.pkl'
    filename = name+'.pkl'
    print('Fitting', filename)
    history = model.fit(
        x_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        verbose=verbose,
        callbacks=[]
    )
    train_loss, train_accuracy = model.evaluate(x_train, y_train, verbose=0)
    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
        pickle.dump(history, file)
    print('========================================================================================')
    print('=', filename)
    print('========================================================================================')
    print(f'= Model loss, accuracy on the train set: {train_loss:.5f} {100*train_accuracy:.2f}')
    print(f'= Model loss, accuracy on the  test set: {test_loss:.5f} {100*test_accuracy:.2f}')
    print('========================================================================================')


### Run Experiments

#### Run single experiment

In [7]:
run_experiment('TWOSIDES_00', 1, 0.25, size, targets, \
               'sigmoid', BinaryCrossentropy(from_logits=False), 0.0001, 'binary_accuracy', \
               x_train, y_train, x_test, y_test, \
               100, 100, 0.2, 1)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 54)                  │          13,878 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,474,678 (17.07 MB)

 Trainable params: 4,474,678 (17.07 MB)

 Non-trainable params: 0 (0.00 B)

None
Fitting TWOSIDES_00.pkl
Epoch 1/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 40ms/step - binary_accuracy: 0.7639 - loss: 0.4794 - val_binary_accuracy: 0.8468 - val_loss: 0.3420
Epoch 2/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 50s 40ms/step - binary_accuracy: 0.8434 - loss: 0.3427 - val_binary_accuracy: 0.8560 - val_loss: 0.3188
Epoch 3/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - binary_accuracy: 0.8586 - loss: 0.3115 - val_binary_accuracy: 0.8529 - val_loss: 0.3206
Epoch 4/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 53s 43ms/step - binary_accuracy: 0.8672 - loss: 0.2946 - val_binary_accuracy: 0.8613 - val_loss: 0.3038
Epoch 5/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 57s 46ms/step - binary_accuracy: 0.8732 - loss: 0.2801 - val_binary_accuracy: 0.8656 - val_loss: 0.2966
Epoch 6/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - binary_accuracy: 0.8783 - loss: 0.2686 - val_binary_accuracy: 0.8664 - val_loss: 0.2920
Epoch 7/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 57s 46ms/step - binary_accuracy: 0.8847 - loss

Epoch 46/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9432 - loss: 0.1288 - val_binary_accuracy: 0.8830 - val_loss: 0.3108
Epoch 47/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9430 - loss: 0.1290 - val_binary_accuracy: 0.8820 - val_loss: 0.3065
Epoch 48/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9438 - loss: 0.1272 - val_binary_accuracy: 0.8835 - val_loss: 0.3091
Epoch 49/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9445 - loss: 0.1262 - val_binary_accuracy: 0.8837 - val_loss: 0.3026
Epoch 50/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9449 - loss: 0.1246 - val_binary_accuracy: 0.8829 - val_loss: 0.3101
Epoch 51/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9453 - loss: 0.1243 - val_binary_accuracy: 0.8845 - val_loss: 0.3082
Epoch 52/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 52s 42ms/step - binary_accuracy: 0.9455 - loss: 0.1241 - val_binary_

1242/1242 ━━━━━━━━━━━━━━━━━━━━ 57s 46ms/step - binary_accuracy: 0.9517 - loss: 0.1092 - val_binary_accuracy: 0.8856 - val_loss: 0.3212
Epoch 92/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 58s 47ms/step - binary_accuracy: 0.9519 - loss: 0.1086 - val_binary_accuracy: 0.8866 - val_loss: 0.3191
Epoch 93/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 57s 46ms/step - binary_accuracy: 0.9525 - loss: 0.1075 - val_binary_accuracy: 0.8862 - val_loss: 0.3236
Epoch 94/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 61s 49ms/step - binary_accuracy: 0.9524 - loss: 0.1075 - val_binary_accuracy: 0.8856 - val_loss: 0.3134
Epoch 95/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 71s 57ms/step - binary_accuracy: 0.9525 - loss: 0.1074 - val_binary_accuracy: 0.8861 - val_loss: 0.3267
Epoch 96/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 63s 50ms/step - binary_accuracy: 0.9523 - loss: 0.1079 - val_binary_accuracy: 0.8875 - val_loss: 0.3199
Epoch 97/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 60s 49ms/step - binary_accuracy: 0.9526 - loss: 0.1073 - val_binary_accuracy: 0.8

#### View Results

In [8]:
def view_results(filename, x_train, y_train, x_test, y_test):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
        history = pickle.load(file)
    print('====================================================================================')
    print('=== Model:', filename)
    print('===')
    if len(x_test) < 100:
        x_test = x_train
        y_test = y_train
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f'=== binary_accuracy {acc:.6f}, loss {loss:.6f}')
    y_pred = model.predict(x_test, verbose=0)
    print(f'=== MSE:      {mean_squared_error(y_test, y_pred):.6f}')
    print(f'=== R2 score: {r2_score(y_test, y_pred):.6f}')
    print('===')
    print('===  Precision - Recall')
    print(f'===     {precision_score(y_test, y_pred.round(), average="macro"):.4f} - \
{recall_score(y_test, y_pred.round(), average="macro"):.4f}')
    for c in range(0, y_pred.shape[1]):
        print(f'=== {c}: {precision_score(y_test[:,c], y_pred.round()[:,c]):.4f} - {recall_score(y_test[:,c], y_pred.round()[:,c]):.4f}')
    return model, history, y_pred

In [9]:
model, history, y_pred = view_results(
    'TWOSIDES_00.pkl', 
    x_train, y_train, x_train, y_train)

=== Model: TWOSIDES_00.pkl
===
=== binary_accuracy 0.946010, loss 0.136599
=== MSE:      0.039301
=== R2 score: 0.727747
===
===  Precision - Recall
===     0.8446 - 0.8380
=== 0: 0.9898 - 0.9776
=== 1: 0.8664 - 0.9508
=== 2: 0.8929 - 0.9779
=== 3: 0.7075 - 0.5451
=== 4: 0.9428 - 0.9811
=== 5: 0.7371 - 0.5086
=== 6: 0.9473 - 0.9814
=== 7: 0.7790 - 0.7340
=== 8: 0.8430 - 0.8386
=== 9: 0.8190 - 0.8234
=== 10: 0.9534 - 0.9820
=== 11: 0.9226 - 0.9770
=== 12: 0.8831 - 0.9481
=== 13: 0.9221 - 0.9801
=== 14: 0.7547 - 0.6709
=== 15: 0.9264 - 0.9796
=== 16: 0.8684 - 0.9421
=== 17: 0.8407 - 0.8908
=== 18: 0.7496 - 0.6453
=== 19: 0.9424 - 0.9810
=== 20: 0.8437 - 0.9336
=== 21: 0.7073 - 0.5068
=== 22: 0.9209 - 0.9774
=== 23: 0.8228 - 0.9263
=== 24: 0.8918 - 0.9625
=== 25: 0.9144 - 0.9769
=== 26: 0.7117 - 0.1353
=== 27: 0.9238 - 0.9792
=== 28: 0.7775 - 0.8913
=== 29: 0.9162 - 0.9767
=== 30: 0.8482 - 0.9346
=== 31: 0.8856 - 0.9551
=== 32: 0.8654 - 0.9456
=== 33: 0.9059 - 0.9709
=== 34: 0.9354 - 0.98